# Test script

In [1]:
import glob
import numpy as np
import nibabel as nib
import nilearn
import os
import pandas as pd
import re

In [3]:
data_loc = os.environ['DATA_LOC']
server_scripts = os.environ['SERVER_SCRIPTS']

In [15]:
regions = ['l_vstr', 'r_vstr']

models = glob.glob(os.path.join(server_scripts, 'rpe_cors/pred_rpes/*.csv'))
models = [os.path.splitext(os.path.basename(x))[0] for x in models]

all_betas = pd.DataFrame()

In [6]:
model = models[0]

In [8]:
beta_img_paths = glob.glob('%s/derivatives/rpe_cors/%s/sub-*/contrasts/sub-*_run-*_*pe_betas.nii.gz'%(data_loc, model))


In [16]:
region = regions[0]

In [22]:
import nilearn.plotting

In [23]:
tmp_img_file_name = beta_img_paths[0]
tmp_img = nib.load(tmp_img_file_name)
mask_file_name = '/oak/stanford/groups/russpold/data/ds000054/0.0.4/derivatives/rois/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w/%s_bin.nii.gz'%(region)
mask = nib.load(mask_file_name)
tmp_img_data = tmp_img.get_fdata()
mask_data = mask.get_fdata()
if tmp_img_data.shape != mask_data.shape:
    mask = nilearn.image.resample_to_img(mask, tmp_img)
    mask_data = mask.get_fdata()
    #binarize resampled mask data
    mask_data = np.where(mask_data >0.1,1,0)
    mask = nilearn.image.new_img_like(tmp_img, mask_data)

In [29]:
cur_beta_img = beta_img_paths[0]
img_data = nib.load(cur_beta_img).get_fdata()
roi_data = np.where(mask_data == 1,img_data,0)
roi_data = roi_data[roi_data != 0]
cur_betas = pd.DataFrame()
cur_betas["beta"] = roi_data
cur_betas["sub_num"] = re.findall('\d+', os.path.basename(cur_beta_img))[0] #take from cur_beta_img
cur_betas["run_num"] = re.findall('\d+', os.path.basename(cur_beta_img))[1] #take from cur_beta_img
cur_betas["pe_type"] = os.path.basename(cur_beta_img).split("_")[2] #take from cur_beta_img
cur_betas["roi"] = region
cur_betas["model"] = model
all_betas = all_betas.append(cur_betas, ignore_index= True)

/home/groups/russpold/software/miniconda/envs/fmri/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [32]:
'%s/derivatives/rpe_cors/all_vstr_betas.csv'%(data_loc)

'/oak/stanford/groups/russpold/data/ds000054/0.0.4/derivatives/rpe_cors/all_vstr_betas.csv'

# Notes on two methods 

## Manual extraction

Extract ROI data manually as in: https://neurostars.org/t/extract-roi-voxels-using-nilearns-niftimasker/3977  

**Won't work if the BOLD does not have the same dimensionality as the mask**

If that's the case, resample the mask

In [ ]:
nilearn.plotting.plot_stat_map(res_mask, colorbar=True, draw_cross=False)

In [ ]:
nilearn.plotting.plot_stat_map(roi_nifti_img, colorbar=True, draw_cross=False)

## Using freesurfer

Extract using freesurfer segmentation as described in https://groups.google.com/forum/#!topic/nipy-user/DjnN9q52vH8

**Not working due to broken freesurfer path**

In [ ]:
import nipype.interfaces.freesurfer as fs 
segstat = fs.SegStats() 
segstat.inputs.segmentation_file = mask_file_name 
segstat.inputs.in_file = img_file_name 
segstat.inputs.avgwf_file = True 
segstat.run() 